In [ ]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file creates a new stacked leaker / nonleaker 
# spectra from the MagE spectra

In [ ]:
import os

import numpy as np
import pandas as pd

from scipy.interpolate import interp1d

import matplotlib.pyplot as plt

In [ ]:
home = os.getcwd()
data = home + '/data'

In [ ]:
df = pd.read_csv(data + '/spectra/mage/slits.txt', delimiter=' ', header=0)

# String array containing slit names
slits = df.iloc[:,0].to_numpy(dtype=str)

# String array containing MAGE spectra filepaths
files = df.iloc[:,1].to_numpy(dtype=str)

# String array containing FIRE spectra filepaths
# fire=df.iloc[:,17].to_numpy(dtype=str)

# Array containing redshifts of each slit
z = df.iloc[:,2].to_numpy(dtype=np.float64)

# Boolean array which indicates if clump leaks LyC
lyc = df.iloc[:,16].to_numpy(dtype=bool)

# Slit magnification factor
mag = df.iloc[:,18].to_numpy(dtype=np.float64)

In [ ]:
nlFiles = ['psz-arcslit-h1-comb1_MWdr.txt', 'psz-arcslit-h3-comb1_MWdr.txt',
    'sunburst_M-6-comb1_MWdr.txt']
lFiles = ['psz-arcslit-h6-comb1_MWdr.txt', 'psz-arcslit-h9-comb1_MWdr.txt', 
    'psz-arcslit-f-comb1_MWdr.txt', 'psz-arcslit-h2-comb1_MWdr.txt']

fileGroups = [nlFiles, lFiles]

stackFiles = ['sba-nonleaker-no_m3.txt', 'sba-leaker-no_m0.txt']

zNL = [2.37067, 2.37045, 2.37067]
zL = [2.37015, 2.37054, 2.37031, 2.37042]

In [ ]:
for i, fileGroup in enumerate (fileGroups):

    for j, file in enumerate(fileGroup):

        #Converting .txt file with MAGE data into a data frame
        df = pd.read_csv(data + '/spectra/mage/' + file, delimiter='\t', header=0, skiprows=13)

        #Discarding missing data
        df = df.apply(pd.to_numeric,errors='coerce').dropna()

        #Discarding extreme outliers
        df = df[df.iloc[:,1] < 1e-20]

        #Resetting the index
        df = df.reset_index(drop=True)

        # Extracting wavelength, flux, and noise
        w = df.iloc[:,0].to_numpy()
        f = df.iloc[:,1].to_numpy() # * 2.998e18 / np.square(w)
        ne = df.iloc[:,2].to_numpy() # * 2.998e18 / np.square(w)
        
        #print(w)

        # Rest-frame wavelengths
        wR = w / (1.0 + z[i])

        # Normalize the flux density by the median flux density
        # between 1500-1600 Å
        ne = ne / np.median(f[(wR >= 1500.0) & (wR <= 1600.0)])
        f = f / np.median(f[(wR >= 1500.0) & (wR <= 1600.0)])

        if j == 0:

            wInt = w[(w >= 3200.5867) & (w <= 8289.4818)]

        func1 = interp1d(w, f)
        func2 = interp1d(w, ne)

        fInt = func1(wInt)
        neInt = func2(wInt)

        if j == 0:

            fT = np.zeros(np.shape(fInt))
            neT = np.zeros(np.shape(neInt))

        fT = fT + fInt
        neT = np.sqrt(((neT)**2) + ((neInt)**2))

    fT = fT / len(fileGroup)
    neT = neT / len(fileGroup)

    #print(np.array([wInt, fT, neT]))

    indices = np.arange(0, len(wInt), 1)
    #print(np.shape(wInt), indices)

    df = pd.DataFrame(np.array([wInt, fT, neT]).T, columns=['wave', 'f', 'noise'], index=indices)
    #print(df)

    df.to_csv(data + '/spectra/mage/' + stackFiles[i], sep='\t', index=False)
    
    '''
    plt.plot(wInt, fT, c='black', ds='steps-mid')
    plt.plot(wInt, neT, c='red', ds='steps-mid')

    plt.xlim(np.min(wInt), np.max(wInt))
    #plt.xlim(1210.0, 1220.0)
    plt.ylim(0.0)

    plt.show()
    '''